Import libraries

In [3]:
import numpy as np
import joblib
import random
import faiss

Load data (with other data this would look different)

In [ ]:
index = faiss.read_index("it.deen.faiss_L2_IVFPQindex_cs64_nprobe32")

In [ ]:
keys = np.memmap("it_train.features.npy", dtype=np.float32, mode='r', shape=(index.ntotal, index.d))
ids = np.memmap("it_train.targets.npy", dtype=np.int32, mode='r', shape=(index.ntotal,))

Create validation set

In [ ]:
num_delete = 50_000
total_in_index = index.ntotal
indices = np.array(random.sample(range(total_in_index), num_delete))

val_keys = keys[indices]
val_ids = ids[indices]

non_indeces = [i for i in range(index.ntotal) if i not in indices]
rest_keys = keys[non_indeces]
rest_ids = ids[non_indeces]

In [ ]:
#Change the datatype to np.arrays
val_keys = np.array(val_keys)
val_ids = np.array(val_ids)
rest_keys = np.array(rest_keys)
rest_ids = np.array(rest_ids)

In [ ]:
def create_index(index_org, new_keys):
    d = index_org.d
    n_list = index_org.nlist
    n_probe = index_org.nprobe
    m = index_org.pq.M
    nbits = index_org.pq.nbits
    quantizer = faiss.IndexFlatL2(d)
    new_index = faiss.IndexIVFPQ(quantizer, d, n_list, m, nbits)
    new_index.nprobe = n_probe
    new_index.train(new_keys)
    new_index.add(new_keys)
    return new_index

In [ ]:
own_index = create_index(index, rest_keys)

Save new data

In [ ]:
#Save validation keys and ids
joblib.dump(val_keys, "validation keys")
joblib.dump(val_ids, "validation ids")

In [ ]:
#Save the index, keys and ids that are not the validation, so what now becomes the original dataset
joblib.dump(rest_keys, "validation keys")
joblib.dump(rest_ids, "validation ids")
faiss.write_index(own_index, "index without validation")